# AI Skills + Google Gemini

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sergioc/ai-skills/blob/main/examples/notebooks/gemini_colab.ipynb)

This notebook demonstrates how to use AI Skills with Google Gemini models.

## Setup

In [ ]:
!pip install aiskills[gemini,search] -q

## Configure API Key

Get your API key from [Google AI Studio](https://makersuite.google.com/app/apikey).

In [ ]:
import os
from getpass import getpass

# In Colab, you can also use Secrets
try:
    from google.colab import userdata
    os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
except:
    pass

if "GEMINI_API_KEY" not in os.environ:
    os.environ["GEMINI_API_KEY"] = getpass("Enter your Gemini API key: ")

## Quick Start: 3 Lines of Code

In [ ]:
from aiskills.integrations import create_gemini_client

client = create_gemini_client()
response = client.chat("Help me debug a memory leak in Python")
print(response)

## How It Works

Gemini's automatic function calling:
1. You send a message to Gemini
2. Gemini recognizes it needs a skill and calls the appropriate function
3. The function executes locally (searches your skills library)
4. Results are automatically fed back to Gemini
5. Gemini generates the final response

## Multi-Turn Chat

In [ ]:
# Start a chat session
chat = client.start_chat()

# First message
response1 = chat.send_message("What skills do you have for testing?")
print("Response 1:")
print(response1.text)
print("\n" + "="*50 + "\n")

# Follow-up
response2 = chat.send_message("Tell me more about the Python debugging one")
print("Response 2:")
print(response2.text)

## Available Tools

The Gemini integration provides these tools:

In [ ]:
# List the tools
tools = client.get_tools()
print("Available tools:")
for tool in tools:
    print(f"  - {tool.__name__}: {tool.__doc__.split(chr(10))[0]}")

## Direct Skill Operations

You can also call skill operations directly:

In [ ]:
# List available skills
skills = client.list_skills()
print(f"Found {len(skills)} skills:\n")
for skill in skills[:5]:
    print(f"  - {skill['name']}: {skill['description'][:50]}...")

In [ ]:
# Search for skills
results = client.search_skills("docker kubernetes")
print(f"Found {results.total} skills:\n")
for r in results.results:
    print(f"  - {r['name']}")

In [ ]:
# Use a skill directly
result = client.use_skill("help me write unit tests in Python")
if result.success:
    print(f"Using skill: {result.skill_name}")
    print(f"\nContent preview:\n{result.content[:500]}...")
else:
    print(f"Error: {result.error}")

## Using Pre-configured Model

For more control, get the model directly:

In [ ]:
from aiskills.integrations import create_gemini_model

# Get a GenerativeModel with tools attached
model = create_gemini_model(model_name="gemini-1.5-flash")  # Faster model

# Use it directly
chat = model.start_chat(enable_automatic_function_calling=True)
response = chat.send_message("How do I optimize database queries?")
print(response.text)

## Manual Function Handling

For complete control over function execution:

In [ ]:
from aiskills.integrations import get_gemini_tools, GeminiSkills
import google.generativeai as genai

# Get tools
tools = get_gemini_tools()

# Create model WITHOUT auto function calling
model = genai.GenerativeModel(
    model_name='gemini-1.5-pro',
    tools=tools,
)

# Start chat without auto-calling
chat = model.start_chat(enable_automatic_function_calling=False)

response = chat.send_message("What skills do you have for Python?")

# Check for function calls
for part in response.parts:
    if hasattr(part, 'function_call'):
        fn = part.function_call
        print(f"Function call: {fn.name}")
        print(f"Arguments: {dict(fn.args)}")
    elif hasattr(part, 'text'):
        print(f"Text: {part.text}")

## Example: Code Review Assistant

In [ ]:
from aiskills.integrations import create_gemini_client

client = create_gemini_client()

code_to_review = '''
def fetch_all_users():
    users = []
    for i in range(10000):
        response = requests.get(f"https://api.example.com/users/{i}")
        if response.status_code == 200:
            users.append(response.json())
    return users
'''

response = client.chat(f"Review this code and suggest improvements:\n```python\n{code_to_review}\n```")
print(response)

## Next Steps

- Try different Gemini models: `gemini-1.5-pro`, `gemini-1.5-flash`
- Install more skills from GitHub
- Create your own skills

For more information, see the [documentation](https://github.com/sergioc/ai-skills).